In [24]:
import numpy as np
import pandas as pd
import os
import sys
from itertools import combinations, product
from datetime import datetime
import numpy_indexed as npi
from scipy.stats import mode
import h5py
from tqdm import tqdm

In [2]:
pid_ls = []
for i in range(150):
    pid_ls.append(i)

In [5]:
combos = combinations(pid_ls, 4)
combo_ls = list(combos)

In [6]:
len(combo_ls)

487635

In [26]:
start = datetime.now()
products = product(pid_ls, repeat = 5)
prod_ls = list(products)
print(len(prod_ls))

print(datetime.now() - start)

24300000
0:00:03.310801


In [28]:
start = datetime.now()
prod_array = np.array(pid_ls)[np.rollaxis(
    np.indices((len(pid_ls),) * 5), 0, 5 + 1)
    .reshape(-1, 5)]

print(len(prod_array))

print(datetime.now() - start)

777600000
0:02:05.617316


In [3]:
def cartesian2(arrays):
    arrays = [np.asarray(a).astype(np.int16) for a in arrays]
    shape = (len(x) for x in arrays)

    ix = np.indices(shape, dtype= np.int16)
    ix = ix.reshape(len(arrays), -1).T

    for n, arr in enumerate(arrays):
        ix[:, n] = arrays[n][ix[:, n]]

    return ix

In [4]:
start = datetime.now()

cart_array = cartesian2([pid_ls, pid_ls, pid_ls, pid_ls])
print(len(cart_array))

print(datetime.now() - start)

506250000
0:00:10.418001


In [5]:
start = datetime.now()
cart_array.sort(axis = 1)
print(len(cart_array))
print(datetime.now() - start)

506250000
0:00:11.995571


In [6]:
start = datetime.now()
cart_array = npi.unique(cart_array)

print(datetime.now() - start)

0:04:55.539809


In [7]:
len(cart_array)

21947850

In [8]:
start = datetime.now()

vals, counts = mode(cart_array, axis = 1)

print(datetime.now() - start)

0:01:32.181373


In [9]:
counts[15789]

array([1])

In [10]:
cart_array[15789]

array([  0,   1,  34, 109], dtype=int16)

In [11]:
counted_array = np.hstack((cart_array, counts))

In [17]:
counted_array[counted_array[:,4] == 1]

array([[  0,   1,   2,   3,   1],
       [  0,   1,   2,   4,   1],
       [  0,   1,   2,   5,   1],
       ..., 
       [145, 146, 148, 149,   1],
       [145, 147, 148, 149,   1],
       [146, 147, 148, 149,   1]])

In [18]:
counted_array = counted_array[counted_array[:,4] == 1]
len(counted_array)

20260275

In [20]:
counted_array = counted_array[:,:4]

In [22]:
test = np.where(np.all(counted_array == [1,2,3,5], axis = 1))
counted_array[test]

array([[1, 2, 3, 5]])

In [37]:
start = datetime.now()
os.chdir('D:/data/lineup_combos')
hdf5 = h5py.File('test_combo_5.h5', 'a')

for batch_start in tqdm(range(80, 150, 5)):
    
    combo_5 = np.zeros(shape = (1,6)).astype(np.int16)
    batch_end = batch_start + 5
    
    for pid in pid_ls[batch_start:batch_end]:
        temp = np.hstack((np.full((len(counted_array), 1), pid), counted_array)).astype(np.int16)

        temp.sort(axis = 1)
        temp = npi.unique(temp)
        
        vals, counts = mode(temp, axis = 1)

        temp_array = np.hstack((temp, counts)).astype(np.int16)
        temp = []
        
        temp_array = temp_array[temp_array[:,5] == 1]

        combo_5 = np.vstack((combo_5, temp_array)).astype(np.int16)

        temp_array = []
    
    
    hdf5.create_dataset('batch_' + str(batch_start), data = combo_5)
    
print(datetime.now() - start)

hdf5.close()


  0%|                                                                                           | 0/14 [00:00<?, ?it/s]C:\Users\vales\Anaconda3\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full((20260275, 1), 80) will return an array of dtype('int32')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)

C:\Users\vales\Anaconda3\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full((20260275, 1), 81) will return an array of dtype('int32')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
C:\Users\vales\Anaconda3\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full((20260275, 1), 82) will return an array of dtype('int32')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
C:\Users\vales\Anaconda3\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full((20260275, 1), 83) will return an array of dtype('int32')
  format(shape, fill

1:52:12.834183


In [39]:
len(combo_5)

98600006